<a href="https://colab.research.google.com/github/dalismo/Finance_News_Sentiment/blob/ab%2Fml_sentiment_test/ab_ml_sentiment_analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [5]:
from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/train-data.csv'

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("train-data.csv"), sep=",", header=None)

df.show(truncate=False)

+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0     |_c1                                                                                                                                                                                                                                                           |
+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|neutral |According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .                                                                 

In [6]:
from pyspark.sql.functions import length
data_df = df.withColumn('length', length(df['_c1']))
data_df.show()

+--------+--------------------+------+
|     _c0|                 _c1|length|
+--------+--------------------+------+
| neutral|According to Gran...|   127|
| neutral|Technopolis plans...|   190|
|negative|The international...|   228|
|positive|With the new prod...|   206|
|positive|According to the ...|   203|
|positive|FINANCING OF ASPO...|   178|
|positive|For the last quar...|   193|
|positive|In the third quar...|   125|
|positive|Operating profit ...|   122|
|positive|Operating profit ...|   103|
|positive|TeliaSonera TLSN ...|   178|
|positive|STORA ENSO , NORS...|   164|
|positive|A purchase agreem...|   207|
|positive|Finnish Talentum ...|   163|
|positive|Clothing retail c...|   143|
|positive|Consolidated net ...|   163|
|positive|Foundries divisio...|   254|
|positive|HELSINKI ( AFX ) ...|   237|
|positive|Incap Contract Ma...|   138|
|positive|Its board of dire...|   125|
+--------+--------------------+------+
only showing top 20 rows



In [19]:
df2 = data_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
df2.printSchema()
df2.show(truncate=False)

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|class   |text                                                                                                                                                                                                                                                          |length|
+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|neutral |According to Gran , the company has no plans to move 

In [22]:
from pyspark.sql.functions import *
newDf = df2.withColumn("text", regexp_replace("text", "[^A-Za-z0-9(),!?@\'\`\"\_\n.]", " "))
newDf.show(truncate=False)

+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|class   |text                                                                                                                                                                                                                                                          |length|
+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|neutral |According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .                                            

In [45]:
import pyspark.sql.functions as f
newDf2 = newDf.withColumn("text",f.lower(f.col("text")))
newDf2.show()

+--------+--------------------+------+
|   class|                text|length|
+--------+--------------------+------+
| neutral|according to gran...|   127|
| neutral|technopolis plans...|   190|
|negative|the international...|   228|
|positive|with the new prod...|   206|
|positive|according to the ...|   203|
|positive|financing of aspo...|   178|
|positive|for the last quar...|   193|
|positive|in the third quar...|   125|
|positive|operating profit ...|   122|
|positive|operating profit ...|   103|
|positive|teliasonera tlsn ...|   178|
|positive|stora enso , nors...|   164|
|positive|a purchase agreem...|   207|
|positive|finnish talentum ...|   163|
|positive|clothing retail c...|   143|
|positive|consolidated net ...|   163|
|positive|foundries divisio...|   254|
|positive|helsinki ( afx ) ...|   237|
|positive|incap contract ma...|   138|
|positive|its board of dire...|   125|
+--------+--------------------+------+
only showing top 20 rows



In [46]:
tokenizer = Tokenizer(inputCol="text", outputCol="Words")

In [47]:
reviewed = tokenizer.transform(newDf2)
reviewed.show()

+--------+--------------------+------+--------------------+
|   class|                text|length|               Words|
+--------+--------------------+------+--------------------+
| neutral|according to gran...|   127|[according, to, g...|
| neutral|technopolis plans...|   190|[technopolis, pla...|
|negative|the international...|   228|[the, internation...|
|positive|with the new prod...|   206|[with, the, new, ...|
|positive|according to the ...|   203|[according, to, t...|
|positive|financing of aspo...|   178|[financing, of, a...|
|positive|for the last quar...|   193|[for, the, last, ...|
|positive|in the third quar...|   125|[in, the, third, ...|
|positive|operating profit ...|   122|[operating, profi...|
|positive|operating profit ...|   103|[operating, profi...|
|positive|teliasonera tlsn ...|   178|[teliasonera, tls...|
|positive|stora enso , nors...|   164|[stora, enso, ,, ...|
|positive|a purchase agreem...|   207|[a, purchase, agr...|
|positive|finnish talentum ...|   163|[f

In [48]:
stopremove = StopWordsRemover(inputCol="Words", outputCol="filtered")

In [49]:
newFrame = stopremove.transform(reviewed)
newFrame.show()

+--------+--------------------+------+--------------------+--------------------+
|   class|                text|length|               Words|            filtered|
+--------+--------------------+------+--------------------+--------------------+
| neutral|according to gran...|   127|[according, to, g...|[according, gran,...|
| neutral|technopolis plans...|   190|[technopolis, pla...|[technopolis, pla...|
|negative|the international...|   228|[the, internation...|[international, e...|
|positive|with the new prod...|   206|[with, the, new, ...|[new, production,...|
|positive|according to the ...|   203|[according, to, t...|[according, compa...|
|positive|financing of aspo...|   178|[financing, of, a...|[financing, aspoc...|
|positive|for the last quar...|   193|[for, the, last, ...|[last, quarter, 2...|
|positive|in the third quar...|   125|[in, the, third, ...|[third, quarter, ...|
|positive|operating profit ...|   122|[operating, profi...|[operating, profi...|
|positive|operating profit .

In [51]:
filtered_df = newFrame.select('filtered')
filtered_df.select("filtered").write.save("filtered.json",format="json")

AnalysisException: ignored

In [52]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
# tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
# stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="filtered", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [53]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [54]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [55]:
cleaner = data_prep_pipeline.fit(newDf2)
cleaned = cleaner.transform(newDf2)

In [56]:
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[32817,33...|
|  0.0|(262145,[20102,21...|
|  2.0|(262145,[4333,939...|
|  1.0|(262145,[29917,32...|
|  1.0|(262145,[161,3968...|
|  1.0|(262145,[3968,811...|
|  1.0|(262145,[2409,538...|
|  1.0|(262145,[33998,38...|
|  1.0|(262145,[9972,339...|
|  1.0|(262145,[33998,38...|
|  1.0|(262145,[35529,45...|
|  1.0|(262145,[9872,201...|
|  1.0|(262145,[4978,126...|
|  1.0|(262145,[33998,38...|
|  1.0|(262145,[1303,531...|
|  1.0|(262145,[11972,22...|
|  1.0|(262145,[9144,997...|
|  1.0|(262145,[2114,688...|
|  1.0|(262145,[24692,45...|
|  1.0|(262145,[12650,21...|
+-----+--------------------+
only showing top 20 rows



In [57]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [58]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show()

+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   class|                text|length|label|               Words|            filtered|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|negative|  net sales for t...|   148|  2.0|[, , net, sales, ...|[, , net, sales, ...|(262144,[5381,105...|(262144,[5381,105...|(262145,[5381,105...|[-703.91074542485...|[1.0,9.7863104860...|       0.0|
|negative|( adp news )   ja...|   177|  2.0|[(, adp, news, ),...|[(, adp, news, ),...|(262144,[43499,45...|(262144,[43499,45...|(262145,[43499,45...|[-1315.7167479499...|[1.0,4.9723757140.

In [59]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.549777
